## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
import logging
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat import EVENT_LOGGER_NAME
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.handlers = [ConsoleLogHandler()]
logger.setLevel(logging.INFO)


In [2]:
from autogenstudio.provider import Provider 
from autogenstudio.teammanager import TeamManager
import json 


team_json_spec = json.load(open("team.json")) 
wm = TeamManager() 

print(team_json_spec)

{'name': 'weather_team', 'participants': [{'name': 'writing_agent', 'model_client': {'model': 'gpt-4o-2024-08-06', 'model_type': 'OpenAIChatCompletionClient'}, 'tools': [{'name': 'get_weather', 'description': 'Get the weather for a city', 'content': 'async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."'}], 'agent_type': 'AssistantAgent'}], 'termination_condition': {'termination_type': 'StopMessageTermination'}, 'team_type': 'RoundRobinGroupChat', 'model_client': None}


In [3]:
result = await wm.run(task="What is the weather in New York?", team_config=team_json_spec)

*** team config *** name='weather_team' participants=[AgentConfig(name='writing_agent', agent_type=<AgentTypes.assistant: 'AssistantAgent'>, system_message=None, model_client=ModelConfig(model='gpt-4o-2024-08-06', model_type=<ModelTypes.openai: 'OpenAIChatCompletionClient'>, api_key=None, base_url=None), tools=[ToolConfig(name='get_weather', description='Get the weather for a city', content='async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."')], description=None)] team_type=<TeamTypes.round_robin: 'RoundRobinGroupChat'> model_client=None termination_condition=TerminationConfig(termination_type=<TerminationTypes.stop_message: 'StopMessageTermination'>, max_messages=None, text=None)

--------------------------------------------------------------------------- 
[2024-11-01T21:34:06.635996]:

What is the weather in New York?
--------------------------------------------------------------------------- 
[2024-11-01T21:34:07.613331], writing_a

In [4]:
result

TeamResult(task_result=TaskResult(messages=[TextMessage(source='user', content='What is the weather in New York?'), ToolCallMessage(source='writing_agent', content=[FunctionCall(id='call_zHJJQKpzMB75O0mhymXzEZmQ', arguments='{"city":"New York"}', name='get_weather')]), ToolCallResultMessages(source='writing_agent', content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_zHJJQKpzMB75O0mhymXzEZmQ')]), TextMessage(source='writing_agent', content='The weather in New York is 73 degrees and sunny.'), StopMessage(source='writing_agent', content='TERMINATE')]), usage='', duration=1.39534592628479)